In [1]:
from astropy.table import Table, join
import numpy as np
import os

In [2]:
def check_consistency(tab1, tab2, joinkey='CATAID', atol=1e-5, path="../../outputs/gama-dmu/"):
    tab1 = Table.read(os.path.join(path, tab1))
    tab2 = Table.read(os.path.join(path, tab2))
    if len(tab1) != len(tab2):
        print(f"Tables are not the same length: {len(tab1)}, {len(tab2)}")
    elif not (tab1[joinkey] == tab2[joinkey]).all():
        print(f"Tables are same length ({len(tab1)}), however joinkey '{joinkey}' not aligned")
        tab1idx = np.sort(tab1[joinkey])
        tab2idx = np.sort(tab2[joinkey])
        if not (tab1idx == tab2idx).all():
            print(f"and joinkeys do not match after sorting")
        else:
            print(f"but joinkeys match after sorting")
    joined = join(tab1, tab2, keys=joinkey, metadata_conflicts='silent')
    for col in tab1.columns:
        if col != joinkey and col in tab2.columns:
            ok = (joined[col+'_1'] == joined[col+'_2']).all()
            if not ok:
                print(f"Duplicate column '{col}' is not identical")
                ok = np.allclose(joined[col+'_1'], joined[col+'_2'], atol=atol)
                if ok:
                    print(f"but is all close")
                else:
                    print(f"and is not all close")

In [3]:
check_consistency("GKGZBinnedv01.fits.gz", "GKGZCleanedv01.fits.gz")

In [4]:
check_consistency("GKGZCleanedv01.fits.gz", "GKGZDebiasedv01.fits.gz")

In [5]:
check_consistency("GKGZDebiasedv01.fits.gz", "GKGZExtrav01.fits.gz")

In [6]:
check_consistency("GKGZDebiasedv01.fits.gz", "GKGZFinalv01.fits.gz")

Tables are same length (49866), however joinkey 'CATAID' not aligned
but joinkeys match after sorting


In [7]:
path = "../../outputs/gama-dmu/"
table = Table.read(os.path.join(path, "GKGZDebiasedv01.fits.gz"))
test = (table['subject_id'] == np.sort(table['subject_id'])).all()
print('Debiased catalolgue sorted by subject_id:', test)

Debiased catalolgue sorted by subject_id: True


In [8]:
path = "../../outputs/gama-dmu/"
table = Table.read(os.path.join(path, "GKGZFinalv01.fits.gz"))
test = (table['CATAID'] == np.sort(table['CATAID'])).all()
print('Final catalolgue sorted by CATAID:', test)

Final catalolgue sorted by CATAID: True
